In [1]:
import os
import time
import html
import re
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select

In [2]:
url = "https://badnet.fr/accueil"
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--window-size=1920x6080')
chromedriver = os.getcwd() + "/chromedriver"

In [3]:
driver = webdriver.Chrome(options = chrome_options, service=Service(chromedriver))
Tournaments = []
tournaments_df = pd.DataFrame({'id':pd.Series(dtype='str'), 
    'name': pd.Series(dtype='str'),
    'url':pd.Series(dtype='str'),
    'departement': pd.Series(dtype='str'), 
    'date': pd.Series(dtype='str'),
    'source': pd.Series(dtype='str'),
    'description': pd.Series(dtype='str'),
    "date_publication": pd.Series(dtype='datetime64[ns]')})

In [4]:
driver.get(url)
time.sleep(10)

affichage = driver.find_element('class name', 'b-views')
affichage.find_elements('tag name','li')[2].click()

time.sleep(3)
print('affichage => liste')

departements = {'75':'62','77':'63','78':'64','91':'65','92':'66','93':'67','94':'68','95':'69'}
for departement, departement_code in departements.items():
    print(departement)
    print("page d'acceuil chargée")
    #ligue_selector = Select(self.driver.find_element("id","ligue"))
    #ligue_selector.select_by_value('12')

    departement_selector = Select(driver.find_element("id","departement"))
    departement_selector.select_by_value(departement_code)
    time.sleep(5)

    

    tournaments=driver.find_element('id', 'search_results').find_elements('class name', 'row')
    print(f"trouvé {len(tournaments)} tournois")
    #print(tournaments.text)
    time.sleep(5)

affichage => liste
75
page d'acceuil chargée
trouvé 5 tournois
77
page d'acceuil chargée
trouvé 7 tournois
78
page d'acceuil chargée
trouvé 4 tournois
91
page d'acceuil chargée
trouvé 7 tournois
92
page d'acceuil chargée
trouvé 4 tournois
93
page d'acceuil chargée
trouvé 5 tournois
94
page d'acceuil chargée
trouvé 7 tournois
95
page d'acceuil chargée
trouvé 4 tournois


In [5]:
tournaments[0].get_attribute("href")

'https://badnet.fr/tournoi/public?eventid=25163'

In [6]:
import os
import time
import re
from datetime import datetime
import pandas as pd
import traceback

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from sqlalchemy import text

#from badnet.sql_connection import badminton_db
#from badnet.utils import logger

class Tournament:
    def __init__(self, name: str, url: str, date: str, ville: str, departement = "99", source="badnet"):
        self.name = name
        self.url = url
        self.id = url.split("=")[-1]
        self.departement = departement
        self.date = date
        self.description = ''
        self.category = [ 'N', 'R', 'D','P', 'NC']
        self.disciplines = ['simple', 'double', 'mixte']
        self.age_group = ['jeunes', 'seniors', 'veterans']
        self.source = source
        self.date_publication = datetime.now()
        self.ville = ville
        self.date_registration_opening = ''
        self.date_registration_closed = ''
        self.ja = 'Non renseigné'
        self.url_affiche='https://badnet.fr/Img/poster/affiche_default.png'
        self.volant = 'Non renseigné'
        self.date_publication = datetime.now()
    
    def __repr__(self) -> str:

        return f"Tournament {self.id} {self.name}"

    def get_additional_info(self):
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--window-size=1920x6080')
        chromedriver = os.getcwd() + "/chromedriver"
        self.driver = webdriver.Chrome(options = chrome_options, service=Service(chromedriver))
        self.driver.get(self.url)
        time.sleep(10)
        self.driver.find_element("xpath", "/html/body/div[2]/div/section/div/div/nav/ul/li[1]/span").click()
        time.sleep(5)
        infos = self.driver.find_element("class name", "infos")
        
        dates = self.driver.find_element("class name", "limit")

        try:
            notes = self.driver.find_element("class name", "text")
            self.description = re.search('Notes des organisateurs\n((.*\n*)*)',notes.text).group(1)
        except Exception as e:
            print(f"Could not extract organizer's notes for {self.name} ID={self.id}")
        ja = re.search('Juge-arbitre : (.+)', infos.text)
        self.ja = ja.group(1) if ja else 'unknown'
        volant = re.search('Volant officiel : (.+)', infos.text)
        self.volant = volant.group(1) if volant else 'unknown'
        
        try:
            simple = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[0]=='check'
            double = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[1]=='check'
            mixte = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[2]=='check'

            discipline = {'simple': simple, 'double': double, 'mixte': mixte}
            self.disciplines = [key for key, value in discipline.items() if value]
            if self.disciplines==[]:
                self.disciplines = ['simple', 'double', 'mixte']
        except Exception as e:
            print(f"Could not extract disciplines for {self.name} ID={self.id}")

        try:
            jeunes = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[0]=='check'
            seniors = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[1]=='check'
            veterans = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[2]=='check'

            age_group = {'jeunes': jeunes, 'seniors': seniors, 'veterans': veterans}
            self.age_group = [key for key, value in age_group.items() if value]
            print(','.join(self.age_group))
            if self.age_group==[]:
                self.age_group = ['jeunes', 'seniors', 'veterans']
        except Exception as e:
            print(f"Could not extract age groups for {self.name} ID={self.id}")

        
        try:
            N = infos.text.split('\n')[-1].split()[0]=='check'
            R = infos.text.split('\n')[-1].split()[1]=='check'
            D = infos.text.split('\n')[-1].split()[2]=='check'
            P = infos.text.split('\n')[-1].split()[3]=='check'
            NC = infos.text.split('\n')[-1].split()[4]=='check'

            categories = {'N': N, 'R': R, 'D': D, 'P': P, 'NC': NC}
            self.category = [key for key, value in categories.items() if value]
            if self.category == []:
                self.category = [ 'N', 'R', 'D','P', 'NC']
        except Exception as e:
            print(f"Could not extract category for {self.name} ID={self.id}")
        try:
            self.date_registration_opening = dates.text.split('\n')[1]
            self.date_registration_closed = dates.text.split('\n')[3]
        except Exception as e:
            print("Could not extract registration dates for {self.name} ID={self.id}")

        self.url_affiche = self.driver.find_element("class name", 'flex.top').find_element("tag name", "figure").find_element("tag name", "a").get_attribute("href")
        
        self.driver.quit()

In [8]:
Tournaments=[]
for tournament in tournaments:
    url = tournament.get_attribute("href")
    name = html.unescape(tournament.find_element('class name', 'name').text)
    date = tournament.find_element('class name', 'date').text
    ville = tournament.find_element('class name', 'location').text
    Tournaments.append(Tournament(name=name, url=url, departement = departement, date=date, ville = ville))
    tournaments_df = pd.concat([pd.DataFrame({
        'id':url.split("=")[-1],
        'name':name,
        'url':url,
        'departement':departement,
        'date':date,
        'source':'badnet',
        'description': '',
        'date_publication':datetime.now()
    }, index=[0]
    ),tournaments_df.loc[:]]).reset_index(drop=True)

In [9]:
Tournaments

[Tournament 25163 BADoVAG5,
 Tournament 25744 Eco-Tournoi Labellisé Jeune LIFB d'Ermont 2024-2025,
 Tournament 25261 Les Mousquetaires fêtent Halloween en étant écoresponsables,
 Tournament 25672 Eco-tournoi de la Grande Horloge]

In [14]:
Tournaments[0].url

'https://badnet.fr/tournoi/public?eventid=25163'

In [11]:
Tournaments[0].get_additional_info()

Could not extract disciplines for BADoVAG5 ID=25163
Could not extract age groups for BADoVAG5 ID=25163


In [13]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--window-size=1920x6080')
chromedriver = os.getcwd() + "/chromedriver"
driver = webdriver.Chrome(options = chrome_options, service=Service(chromedriver))
url=Tournaments[0].url
driver.get(url)
time.sleep(10)
driver.find_element("xpath", "/html/body/div[2]/div/section/div/div/nav/ul/li[1]/span").click()
time.sleep(5)
infos_tt = driver.find_elements("class name", "infos")
titre = infos_tt[0].find_element('tag name', 'h2').text
if titre == 'Informations pratiques':
    infos = infos_tt[0]
else:
    infos = infos_tt[len(infos_tt)-1]


dates = driver.find_element("class name", "limit")

try:
    notes = driver.find_element("class name", "text")
    description = re.search('Notes des organisateurs\n((.*\n*)*)',notes.text).group(1)
except Exception as e:
    print(f"Could not extract organizer's notes for")
ja = re.search('Juge-arbitre : (.+)', infos.text)
ja = ja.group(1) if ja else 'unknown'
volant = re.search('Volant officiel : (.+)', infos.text)
volant = volant.group(1) if volant else 'unknown'

try:
    simple = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[0]=='check'
    double = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[1]=='check'
    mixte = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[2]=='check'

    discipline = {'simple': simple, 'double': double, 'mixte': mixte}
    disciplines = [key for key, value in discipline.items() if value]
    if disciplines==[]:
        disciplines = ['simple', 'double', 'mixte']
except Exception as e:
    print(f"Could not extract disciplines for ")

try:
    jeunes = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[0]=='check'
    seniors = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[1]=='check'
    veterans = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[2]=='check'

    age_group = {'jeunes': jeunes, 'seniors': seniors, 'veterans': veterans}
    age_group = [key for key, value in age_group.items() if value]
    print(','.join(age_group))
    if age_group==[]:
        age_group = ['jeunes', 'seniors', 'veterans']
except Exception as e:
    print(f"Could not extract age groups for")


try:
    N = infos.text.split('\n')[-1].split()[0]=='check'
    R = infos.text.split('\n')[-1].split()[1]=='check'
    D = infos.text.split('\n')[-1].split()[2]=='check'
    P = infos.text.split('\n')[-1].split()[3]=='check'
    NC = infos.text.split('\n')[-1].split()[4]=='check'

    categories = {'N': N, 'R': R, 'D': D, 'P': P, 'NC': NC}
    category = [key for key, value in categories.items() if value]
    if category == []:
        category = [ 'N', 'R', 'D','P', 'NC']
except Exception as e:
    print(f"Could not extract category for ")
try:
    date_registration_opening = dates.text.split('\n')[1]
    date_registration_closed = dates.text.split('\n')[3]
except Exception as e:
    print("Could not extract registration dates for")

url_affiche = driver.find_element("class name", 'flex.top').find_element("tag name", "figure").find_element("tag name", "a").get_attribute("href")

Could not extract organizer's notes for
seniors


In [14]:
date_registration_opening

'09/08/2023'

In [38]:
infos = driver.find_elements("class name", "infos")


In [66]:
url = Tournaments[0].url
name = Tournaments[0].name
id = Tournaments[0].id


chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--window-size=1920x6080')
chromedriver = os.getcwd() + "/chromedriver"
driver = webdriver.Chrome(options = chrome_options, service=Service(chromedriver))
driver.get(url)
time.sleep(10)
driver.find_element("xpath", "/html/body/div[2]/div/section/div/div/nav/ul/li[1]/span").click()
time.sleep(5)
infos_tt = driver.find_elements("class name", "infos")
titre = infos_tt[0].find_element('tag name', 'h2').text
if titre == 'Informations pratiques':
    infos = infos_tt[0]
else:
    infos = infos_tt[len(infos_tt)-1]

dates = driver.find_element("class name", "limit")

try:
    notes = driver.find_element("class name", "text")
    description = re.search('Notes des organisateurs\n((.*\n*)*)',notes.text).group(1)
except Exception as e:
    print(f"Could not extract organizer's notes for {name} ID={id}")
    print(traceback.format_exc())

ja = re.search('Juge-arbitre : (.+)', infos.text)
ja = ja.group(1) if ja else 'unknown'
volant = re.search('Volant officiel : (.+)', infos.text)
volant = volant.group(1) if volant else 'unknown'

try:
    simple = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[0]=='check'
    double = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[1]=='check'
    mixte = infos.text.split('\n')[infos.text.split('\n').index('SIMPLE DOUBLE MIXTE')+1].split()[2]=='check'

    discipline = {'simple': simple, 'double': double, 'mixte': mixte}
    disciplines = [key for key, value in discipline.items() if value]
    if disciplines==[]:
        disciplines = ['simple', 'double', 'mixte']
except Exception as e:
    print(f"Could not extract disciplines for {name} ID={id}")
    print(traceback.format_exc())


try:
    jeunes = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[0]=='check'
    seniors = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[1]=='check'
    veterans = infos.text.split('\n')[infos.text.split('\n').index('JEUNES SÉNIORS VÉTÉRANS HANDIBAD INCLUSIF')+1].split()[2]=='check'

    age_group = {'jeunes': jeunes, 'seniors': seniors, 'veterans': veterans}
    age_group = [key for key, value in age_group.items() if value]
    print(','.join(age_group))
    if age_group==[]:
        age_group = ['jeunes', 'seniors', 'veterans']
except Exception as e:
    print(f"Could not extract age groups for {name} ID={id}")
    print(traceback.format_exc())


try:
    classements = infos.text.split('\n')[-1]
    N = 'N' in classements #=='check'
    R = 'R' in classements #=='check'
    D = 'D' in classements
    P = 'P' in classements
    NC = 'check' in classements

    list_classements = [x for x in classements.split() if len(x) in (2,3)]

    categories = {'N': N, 'R': R, 'D': D, 'P': P, 'NC': NC}
    category = [key for key, value in categories.items() if value]
    if category == []:
        category = [ 'N', 'R', 'D','P', 'NC']
except Exception as e:
    print(f"Could not extract category for {name} ID={id}")
    print(traceback.format_exc())
try:
    date_registration_opening = dates.text.split('\n')[1]
    date_registration_closed = dates.text.split('\n')[3]
except Exception as e:
    print("Could not extract registration dates for {self.name} ID={self.id}")

#driver.quit()

seniors


In [67]:
[x for x in classements.split() if len(x) in (2,3)]

['R4', 'R5', 'R6', 'D7', 'D8', 'D9', 'P10', 'P11', 'P12']

In [65]:
classements

'close R4 R5 R6 D7 D8 D9 P10 P11 P12 check'

In [49]:
R

'R4'

In [50]:
D

'R5'

In [51]:
P

'R6'

In [52]:
NC

'D7'